## Script para la construcción de las tipologías de continuos poblados
La tipología de continuos poblados necesita los shapes de continuos poblados sin recortar; es decir, donde la áreas de los continuos se **superpongan** unos con otros.

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd 
                        
import os

##### Abrimos las bases de datos, toda se encuentran en el mismo sistema de coordenadasa

In [65]:
# Definimos la ruta de los archivos
path_d=r"C:\Users\Guillermo\Desktop/Python\01. Continuo urbano-rural"
## BD de continuos poblados por densidad
np_3=gpd.read_file(path_d+'\\02. Continuos poblados_res\\01_Conglomerado_alta_densidad.shp')
np_2=gpd.read_file(path_d+'\\02. Continuos poblados_res\\02_Conglomerado_media_densidad.shp')
np_1=gpd.read_file(path_d+'\\02. Continuos poblados_res\\03_Conglomerado_baja_densidad.shp')

## Verificamos el sistema de coordenadas
print(np_1.crs)
print(np_2.crs)
print(np_3.crs)

epsg:32718
epsg:32718
epsg:32718


In [66]:
## Seleccionamos solo las variables que nos interesan
np_1r=np_1[np_1.id_n.notna()][['id_n','geometry']].copy().rename({'id_n':'idnp'}, axis=1)
np_2r=np_2[np_2.id_n.notna()][['id_n','geometry']].copy().rename({'id_n':'idnp'}, axis=1)
np_3r=np_3[np_3.id_n.notna()][['id_n','geometry']].copy().rename({'id_n':'idnp'}, axis=1)

## creamos una BD que solo contenga la variable de idnp_zona
# np_f_r=np_f[['id_contpb']].copy()
# np_f_r['zona']=np_f_r.id_contpb.str[:1]

##### Creamos la lista de continuos poblados que desaparecen al no tener población
Este paso solo aplica si la tipología se realiza para los continuo poblados con población. Este paso se debe omitir si se desea tener una tipología solo para los polígonos de las tres densidades independientemente de que dichos polígonos cuenten con población según el CPV 2017

In [7]:
# ### Cruzamos la BD originales con la base limpia para obtener los conti
# np_3r1=np_3r.merge(np_f_r[np_f_r.zona=='A'], left_on='idnp', right_on='id_contpb',how='outer',  validate='1:1', indicator=True)
# np_3c=np_3r1[np_3r1._merge=='both'].copy()

# np_1r1=np_1r.merge(np_f_r, left_on='idnp', right_on='id_contpb',how='outer',  validate='1:1', indicator=True)
# np_1c=np_1r1[np_1r1._merge=='both'].copy()

# np_15r1=np_15r.merge(np_f_r, left_on='idnp', right_on='id_contpb',how='outer',  validate='1:1', indicator=True)
# np_15c=np_15r1[np_15r1._merge=='both'].copy()

# ## Nos quedamos únicamente con las dos variables que nos interesan: el código del continuo poblado y su geometría para
# ## los geoprocesamientos
# np_1r=np_1c[['idnp','geometry']].copy()
# np_2r=np_2c[['idnp','geometry']].copy()
# np_3r=np_3c[['idnp','geometry']].copy()

NameError: name 'np_f_r' is not defined

## Geo procesamiento
- p1: continuos poblados de 0.8 viv/ha que contienen continuos poblados de 3.9 viv ha.
- p2: continuos poblados de 0.13 viv/ha que contienen continuos poblados de 3.9 viv ha.
- p3: continuos poblados de 0.13 viv/ha que contienen continuos poblados de 0.8 viv ha.

##### Mediana densidad

In [73]:
p1 = gpd.sjoin(np_2r, np_3r, how="left", predicate='intersects')
### Calculo de la número de continuos poblados y de las donnas
p1['n3_2']=np.where(p1.index_right.notna(),1,0) ##toma valor 1 cuando encuentra algún polígono de densidad 3.9 en el polígono de densidad 0.8

p1=p1.groupby('idnp_left').sum().reset_index().sort_values('n3_2') ## se suma el número de continuos de densidad 3.9 en el polígono de densidad 0.8
p1['dum3_2']=np.where(p1.n3_2>=1,1,0) ## Creamos una variable dummy 1 sí continene un continuo poblado de densidad 3; 0, no contiene un continuo poblado de densidad 3

# # ## Nos quedamos con las variables que nos interesan 
p1=p1[['idnp_left','n3_2','dum3_2']].copy()
p1

,idnp_left,n3_2,dum3_2
0,1,0,0
161,162,0,0
162,163,0,0
163,164,0,0
164,165,0,0
...,...,...,...
173,174,1,1
167,168,1,1
160,161,1,1
248,249,1,1


##### Baja densidad

In [78]:
##Baja densidad con alta densidad
p2 = gpd.sjoin(np_1r, np_3r, how="left", predicate='intersects')
# Calculo de la número de continuos poblados y de las donnas
p2['n3_1']=np.where(p2.index_right.notna(),1,0)
p2=p2.groupby('idnp_left').sum().reset_index().sort_values('n3_1')
p2['dum3_1']=np.where(p2.n3_1>=1,1,0)
p2=p2[['idnp_left','n3_1','dum3_1']].copy()
p2

,idnp_left,n3_1,dum3_1
0,1,0,0
34170,34171,0,0
34171,34172,0,0
34172,34173,0,0
34173,34174,0,0
...,...,...,...
26270,26271,1,1
38151,38152,1,1
19821,19822,1,1
18309,18310,2,1


In [82]:
##Baja densidad con media densidad
p3 = gpd.sjoin(np_1r, np_2r, how="left", predicate='intersects')
# Calculo de la número de continuos poblados y de las donnas
p3['n2_1']=np.where(p3.index_right.notna(),1,0)
p3=p3.groupby('idnp_left').sum().reset_index().sort_values('n2_1')
p3['dum2_1']=np.where(p3.n2_1>=1,1,0)
p3=p3[['idnp_left','n2_1','dum2_1']].copy()
p3

,idnp_left,n2_1,dum2_1
0,1,0,0
34156,34157,0,0
34157,34158,0,0
34159,34160,0,0
34160,34161,0,0
...,...,...,...
26461,26462,3,1
19821,19822,3,1
28141,28142,3,1
15778,15779,4,1


##### Juntamos las posibilidades de continuos poblados de baja densidad y creamos una tipología

In [83]:
p15_f=p3.merge(p2, on='idnp_left', how='outer', validate='1:1')
p15_f

,idnp_left,n2_1,dum2_1,n3_1,dum3_1
0,1,0,0,0,0
1,34157,0,0,0,0
2,34158,0,0,0,0
3,34160,0,0,0,0
4,34161,0,0,0,0
...,...,...,...,...,...
51268,26462,3,1,0,0
51269,19822,3,1,1,1
51270,28142,3,1,5,1
51271,15779,4,1,1,1


In [84]:
## Creamos la tipología de continuos poblados
p15_f['tipologia']='B0'
p15_f['tipologia']=np.where((p15_f.dum1_2==1)&(p15_f.n1_2==1),'B1',p15_f.tipologia)
p15_f['tipologia']=np.where((p15_f.dum1_2==1)&(p15_f.n1_2>1),'B2',p15_f.tipologia)
p15_f['tipologia']=np.where((p15_f.dum1_3==1)&(p15_f.n1_3==1),'C1',p15_f.tipologia)
p15_f['tipologia']=np.where((p15_f.dum1_3==1)&(p15_f.n1_3>1),'C2',p15_f.tipologia)

p15_f['tipologia']=np.where((p15_f.dum1_2==1)&(p15_f.n1_2==1)&(p15_f.dum1_3==1)&(p15_f.n1_3==1),'D1',p15_f.tipologia)
p15_f['tipologia']=np.where((p15_f.dum1_2==1)&(p15_f.n1_2==1)&(p15_f.dum1_3==1)&(p15_f.n1_3>1),'D2',p15_f.tipologia)
p15_f['tipologia']=np.where((p15_f.dum1_2==1)&(p15_f.n1_2>1)&(p15_f.dum1_3==1)&(p15_f.n1_3==1),'D3',p15_f.tipologia)
p15_f['tipologia']=np.where((p15_f.dum1_2==1)&(p15_f.n1_2>1)&(p15_f.dum1_3==1)&(p15_f.n1_3>1),'D4',p15_f.tipologia)

p1['tipologia']='E0'
p1['tipologia']=np.where((p1.dum2_3==1)&(p1.n2_3==1),'E1',p1.tipologia)
p1['tipologia']=np.where((p1.dum2_3==1)&(p1.n2_3>1),'E2',p1.tipologia)

AttributeError: 'DataFrame' object has no attribute 'dum1_2'

In [55]:
p15_f

,idnp_left,n1_2,dum1_2,n1_3,dum1_3,tipologia
0,1,1,1,1,1,D1
1,25,1,1,1,1,D1
2,26,1,1,1,1,D1
3,27,1,1,1,1,D1
4,28,1,1,1,1,D1
5,29,1,1,1,1,D1
6,30,1,1,1,1,D1
7,31,1,1,1,1,D1
8,32,1,1,1,1,D1
9,33,1,1,1,1,D1


##### Todas los continuos poblados de baja densidad con la variable de tipología
Obj: Agregar los continuos poblados de Media densidad no incluidos y agregar los continuos poblados de Alta densidad no incluidos


In [27]:
## Todos los NP de baja densidad con la variable de tipología 
np_15r2=np_15r.merge(p15_f, left_on='idnp', right_on='idnp_left',
                     how='outer', validate='1:1', indicator=True)[['idnp_left','geometry','n15_1','dum15_1','n15_3','dum15_3','tipologia']] ##match perfecto
np_1r2=np_1r.merge(p1, left_on='idnp', right_on='idnp_left',
                   how='outer', validate='1:1', indicator=True)[['idnp_left','geometry','n1_3','dum1_3','tipologia']] ##match perfecto

Hacemos un geoprocesamiento para identificar a los continuos poblados de MEDIANA densidad que se encuentran incluidos en la BD de BAJA densidad,
luego recodificamos las variable "tipología"

In [33]:
np_15r3[np_15r3.index_left.notna()]

,index_left,tipologia_left,idnp_left,geometry,n1_3,dum1_3,tipologia_right
0,101,D3,48.0,"POLYGON ((1000427.900 7992427.000, 1000402.900...",1,1,E1
1,101,D3,51.0,"POLYGON ((998477.900 7995427.000, 998452.900 7...",0,0,E0
2,204,D1,97.0,"POLYGON ((893027.900 8043127.000, 893002.900 8...",1,1,E1
3,452,D1,199.0,"POLYGON ((931527.900 8091777.000, 931502.900 8...",1,1,E1
4,524,B1,205.0,"POLYGON ((841127.900 8097927.000, 841102.900 8...",0,0,E0
...,...,...,...,...,...,...,...
289,51075,B1,25052.0,"POLYGON ((-133822.100 9589577.000, -133847.100...",0,0,E0
290,51151,B1,25095.0,"POLYGON ((-124022.100 9594777.000, -124047.100...",0,0,E0
291,51158,B1,25102.0,"POLYGON ((-107422.100 9595877.000, -107447.100...",0,0,E0
292,51201,D1,25143.0,"POLYGON ((-106322.100 9600827.000, -106347.100...",1,1,E1


In [34]:
np_15r3=gpd.sjoin(np_15r2[['tipologia','geometry']], np_1r2, how="right", op='intersects')
np_15r3['tipologia_left']=np.where(np_15r3.index_left.isna(),np_15r3.tipologia_right, np_15r3.tipologia_left)
np_15r3=np_15r3.reset_index()[['idnp_left','geometry','n1_3','dum1_3','tipologia_left']].copy()
np_15r3.rename({'tipologia_left':'tipologia'},axis=1, inplace=True)

g1=gpd.sjoin(np_15r2[['tipologia','geometry']], np_3r, how='right', op='intersects').reset_index()
g1['tipologia']=np.where(g1.index_left.isna(),'A0',g1.tipologia)
g1=g1[['idnp','geometry','tipologia']].copy()
g1.rename({'idnp':'idnp_left'},axis=1, inplace=True)


C:\Users\Guillermo\anaconda3\envs\geo1\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\Guillermo\anaconda3\envs\geo1\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


##### Juntamos la BD de BAJA, MEDIA y ALTA densidad 

In [35]:
np_f1=np_15r2.append(np_15r3)
#np_ft=np_f1.append(g1)

In [36]:
##ALTA Y MEDIA DENSIDAD
np_f1=g1.append(np_15r3)
np_f1=np_f1[['idnp_left','tipologia']].copy()
##grabamos
np_f1.to_excel(o+'\\a_m_tipologia.xlsx', index=False)

##BAJA DENSIDAD
np_f2=np_15r2
np_f2=np_f2[['idnp_left','tipologia']].copy()
##grabamos
np_f2.to_excel(o+'\\baja_tipologia.xlsx', index=False)

In [37]:
np_f2.groupby('tipologia').size()

tipologia
B0    51644
B1      208
B2       11
D1       27
D3       11
D4        2
dtype: int64

In [38]:
np_f1.groupby('tipologia').size()

tipologia
B1    208
B2     29
D1     54
D3     36
D4     12
dtype: int64